In [1]:
# set current working directory
import os
if os.name == 'posix':
    %cd ~/workspace/python/CNStocks
else:
    %cd D:\workspace\python\CNStocks
        
import h5py

from types import SimpleNamespace
%load_ext Cython

D:\workspace\python\CNStocks


In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [ ]:
msl = SimpleNamespace()
fsl = SimpleNamespace()
stat = SimpleNamespace()
basics = SimpleNamespace()
file = os.path.join(os.getcwd(), 'hdf5', '20190808.hdf5')

with h5py.File(file, "a") as f:
    codes = np.copy(np.char.decode(f[u'codes'], 'utf-8'))
#     setup_arrays(codes)

    names = np.copy(np.char.decode(f[u'names'], 'utf-8'))
    msl.data = np.copy(f[u'msl'][:,:,:])
    fsl.data = np.copy(f[u'fsl'][:,:,:])
    stat.data = np.copy(f[u'stat'][:,:,:])
    basics.data = np.copy(f[u'basics'][:,:])

In [ ]:
index_ms = 2999
index_fs = 249

In [ ]:
ms = msl.data[index_ms, :, :]

In [ ]:
fs = fsl.data[index_fs, :, :]

In [ ]:
b = basics.data[:, :]

In [ ]:
st = stat.data[index_fs, :, :]

In [ ]:
index_fs5p = min(index_fs - 5, 9)
fs5p = fsl.data[index_fs5p, :, :]
fs5p[:,0] = fs[:,0]*0.95

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport isnan

ctypedef np.float32_t DTYPE_t

@cython.boundscheck(False)
@cython.cdivision
@cython.wraparound(False)
cpdef compute_stats(DTYPE_t[:,:] ms, DTYPE_t[:,:] fs, DTYPE_t[:,:] b, DTYPE_t[:,:] st, DTYPE_t[:,:] fs5p, Py_ssize_t index_fs):
    
    # ms
    cdef Py_ssize_t idx_open = 0
    cdef Py_ssize_t idx_now = 2
    cdef Py_ssize_t idx_close = 1
    cdef Py_ssize_t idx_turnover = 5
    cdef Py_ssize_t idx_volume = 6
    
    cdef Py_ssize_t idx_zf = 0
    cdef Py_ssize_t idx_jj = 1
    cdef Py_ssize_t idx_lb = 2
    cdef Py_ssize_t idx_zs = 3
    cdef Py_ssize_t idx_zt = 4
    cdef Py_ssize_t idx_fsto = 5
    
    cdef Py_ssize_t rows = ms.shape[0]
    cdef Py_ssize_t cols = ms.shape[1]
    
    cdef Py_ssize_t interval = min(index_fs - 9, 1)
    
    cdef list indices_zt = []
    for i in range(rows):
        if ms[i,idx_open]:
            st[i,idx_zf] = 100*(ms[i,idx_now]/ms[i,idx_close]-1)

        if ms[i,idx_turnover]:
            st[i,idx_jj] = ms[i,idx_volume]/ms[i,idx_turnover]
            st[i,idx_zs] = 100*(ms[i,idx_now]/fs5p[i, 0]-1)
            if ms[i,idx_now] - b[i,0] > -0.0001:
                indices_zt.append(i)

        if b[i,2] and ms[i,idx_turnover]:
            st[i,idx_lb] = ms[i,idx_turnover]/interval/b[i,2]

    return None

In [ ]:
%%timeit -n1000
compute(ms, fs, b, st, fs5p, index_fs)

In [ ]:
import numpy as np

In [ ]:
st.fill(np.nan)
st

In [ ]:
st

In [ ]:
ms

In [ ]:
names

In [ ]:
np.get_include()

In [3]:
from libs.cython.compute import compute_stats